<a href="https://colab.research.google.com/github/ggyppsyy/colab_experiments/blob/master/Jax_DC_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Sources
#https://wiseodd.github.io/techblog/2016/07/16/convnet-conv-layer/
#https://github.com/huyouare/CS231n/blob/master/assignment2/cs231n/im2col.py
#https://colab.research.google.com/github/google/jax/blob/master/notebooks/neural_network_and_data_loading.ipynb

In [0]:
#!pip install --upgrade -q https://storage.googleapis.com/jax-wheels/cuda$(echo $CUDA_VERSION | sed -e 's/\.//' -e 's/\..*//')/jaxlib-0.1.12-cp36-none-linux_x86_64.whl
#!pip install --upgrade -q jax

In [0]:
import numpy as np

In [0]:
#Requires shape (batch, height, width, layers)
def conv_forward(X, W, b, stride=1, padding=1):
    cache = 
    n_filters, h_filter, w_filter, d_filter = W.shape
    n_x, h_x, w_x, d_x = X.shape
    assert (h_x - h_filter + 2 * padding) % stride == 0
    assert (w_x - w_filter + 2 * padding) % stride == 0
    h_out = (h_x - h_filter + 2 * padding) / stride + 1
    w_out = (w_x - w_filter + 2 * padding) / stride + 1
    
    

[[[ 0  1  2]
  [ 3  4  5]]

 [[ 6  7  8]
  [ 9 10 11]]]
(2, 2, 3)
[[[ 0  3]
  [ 1  4]
  [ 2  5]]

 [[ 6  9]
  [ 7 10]
  [ 8 11]]]
(2, 3, 2)
